# Fetch Rewards Coding Project
## Overview
In this project, I will demonstrate my ability to reason about data as well as showcase how I communicate findings and insights with stakeholders and business executives.

----
## Requirements
1. Review unstructured JSON data and diagram a new structured relational data model

2. Generate a query that answers a predetermined business question
    - What are the top 5 brands by receipts scanned for most recent month?
    - How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?
    - When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
    - When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
    - Which brand has the most spend among users who were created within the past 6 months?
    - Which brand has the most transactions among users who were created within the past 6 months?
>     
3. Generate a query to capture data quality issues against the new structured relational data model

4. Write a short email or Slack message to the business stakeholder (Found outside of notebook)
    - What questions do you have about the data?
    - How did you discover the data quality issues?
    - What do you need to know to resolve the data quality issues?
    - What other information would you need to help you optimize the data assets you're trying to create?
    - What performance and scaling concerns do you anticipate in production and how do you plan to address them?

---
## The Data

### Receipts Data Schema

Column | Definition
--- | -----------
_id | uuid for this receipt
bonusPointsEarned | Number of bonus points that were awarded upon receipt completion
createDate | The date that the event was created
dateScanned | Date that the user scanned their receipt
finishedDate | Date that the receipt finished processing
modifyDate | The date the event was modified
pointsAwardedDate | The date we awarded points for the transaction
pointsEarned | The number of points earned for the receipt
purchaseDate | The date of the purchase
purchasedItemCount | Count of number of items on the receipt
rewardsReceiptItemList | The items that were purchased on the receipt
rewardsReceiptStatus | Status of the receipt through receipt validation and processing
totalSpent | The total amount on the receipt
userId | String id back to the User collection for the user who scanned the receipt

----
----

### Users Data Schema

Column | Definition
--- | -----------
_id | User Id
state | state abbreviation
createdDate | When the user created their account
lastLogin | Last time the user was recorded logging in to the app
role | Constant value set to 'CONSUMER'
active | Indicates if the user is active; only Fetch will de-activate an account with this flag

----
----

### Brand Data Schema

Column | Definition
--- | -----------
_id | Brand uuid
barcode | The barcode on the item
brandCod | String that corresponds with the brand column in a partner product file
category | The category name for which the brand sells products in
categoryCode | The category code that references a BrandCategory
cpg | Reference to CPG collection
topBrand | Boolean indicator for whether the brand should be featured as a 'top brand'
name |Brand name

---

In [43]:
import gzip
import json
import shutil
import os
from ast import literal_eval
import pandas as pd
from datetime import datetime
import pyodbc
import sqlalchemy as sal
import numpy as np

In [44]:
for f in os.listdir(): # For unzipping .gz files 
  if 'json' in f:  
      with gzip.open(f, 'rb') as f_in:
          with open(f.replace('.gz',''), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

---
# Part One: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model

In this section, we will be formatting our JSON files, processing nested JSON objects, specifying our column datatypes, and inserting our data into SQL Server. 

## File Formatting

Due to invalid formatting found in the JSON files during data modeling, the code below assist us in validating and cleaning JSON data for processing. First, run *python -m json.tool filename* in the command line to check whether the file is a valid JSON document. You should receive an error if invalid. Othewise, the whole file prints. 

If an error is confirmed, run the data pipeline below to render a clean json file. The data pipeline is defined to utilze the JSONDecoder.raw_decode() (and its undocumented second parameter) to traverse the data, look for valid JSON structures in an iterative manner, and parse any invalid structures it encounters. A nice benefit to this built-in json module is that it will properly parse the data even if the concatenated JSONs are not properly indented or are just missing. 

Once all our JSON data has been parsed, the file will be outputted, read again, and unnested at the first level. This should aid us in idenitifying which JSON objects need to be flatten even further.

*Please note that even after running the JSON files into the data pipeline, the data will still be structured as a JSON array (or list in Python) rather than the standard JSON object (or dict in Python)*

In [45]:
def jsonFormatter(filename, parsed= None, parser= None): 
    parser = json.JSONDecoder() 
    parsed = [] # a list to hold individually parsed JSON structures
    with open('{filename}.json'.format(filename = filename)) as f: 
        data = f.read() 
        head = 0 # hold the current position as we parse while True: 
        while True:
            head = (data.find('{', head) + 1 or data.find('[', head) + 1) - 1
            try:
                struct, head = parser.raw_decode(data, head)
                parsed.append(struct)
            except (ValueError, json.JSONDecodeError):  # no more valid JSON structures
                break

    with open('{filename}Clean.json'.format(filename = filename), 'w', encoding='utf-8') as jsonfile: # Parsed file is outputted for documentation
        json.dump(parsed, jsonfile, ensure_ascii=False, indent=2)

        df = pd.json_normalize(parsed, max_level = 1) # objects unnested
        df.rename(columns=lambda x: x.split('.')[0].replace(' ','') if '.' in x else x, inplace= True) #removing json keys in column name
        return df

In [46]:
# Load fetch rewards datasets.
users = jsonFormatter('users')
receipts = jsonFormatter('receipts')
brands = jsonFormatter('brands')

## Flattening Deeply Nested JSON Objects

After formatting our datasets into a desired state, our next step is to flatten the deeply nested objects and extract the JSON arrays (or lists) that are embedded within each key. From there, we can ensure we have access to the values that will inform our analysis later on. 

### Receipts

 In order to access the remaining items nested in the Receipts dataset, we need to explode 'rewardsReceiptItemList' so we can access the lists of receipts. From there, we ensure all the values, especially the NAs, are embedded within the lists so that we can convert them into strings and then feed them into our literal_eval function. Finally, after detecting each dictionary and list,  we run json_normalize to unnest all keys and values and merge them back to their respective datasets by index. In the end, each 'receiptId' should have duplicated rows that represent each individual item by receipt. This was done to more easily extract brand and item information. 

In [47]:
receipts = receipts.reindex(sorted(receipts.columns), axis=1) 

In [48]:
receipts = receipts.explode('rewardsReceiptItemList') # explode nested objects
receipts.reset_index(inplace=True)

receipts = receipts.fillna({'rewardsReceiptItemList':'{}'}) # adding curly bracklets to detect lists among NAs
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x)) # converting to strings
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval) # detecting dictionaries and lists

In [49]:
rewardsReceiptsFlat = pd.json_normalize(receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList') # unnesting by variable, ideally performed with meta
rewardsReceiptsFlat.rename(columns =  {'pointsEarned': 'pointsEarnedReceipt'}, inplace= True) # to avoid duplication
rewardsReceiptsFlat = rewardsReceiptsFlat.reindex(sorted(rewardsReceiptsFlat.columns), axis=1) 

In [50]:
receiptsClean =  pd.merge(receipts, rewardsReceiptsFlat, left_index = True, right_index = True, how = 'outer') # Merging by index

In [51]:
receiptsClean = receiptsClean.drop(['rewardsReceiptItemList', 'index'], axis= 1)
receiptsClean.rename(columns = {'_id':'receiptId'}, inplace= True)

In [52]:
def strip_obj(col): # Function to remove whitespace
    if col.dtypes == object:
        return (col.astype(str)
                   .str.strip()
                   .replace({'nan': np.nan}))
    return col

In [53]:
receiptsClean = receiptsClean.apply(strip_obj, axis=0)

In [54]:
receiptsClean = receiptsClean.dropna(axis = 1, how = 'all')
receiptsClean = receiptsClean.drop_duplicates()

## Brands

As a reult of the file formatting, the 'cpg' key was unneested, however, the values for the keys 'id' and 'ref' are now contained in two separate columns with duplicated column names, 'cpg'. To access the values, all we need to do is differentiate the columns and unnest the remaining key, 'oid', which er will merge back in by index. 

In [55]:
s = pd.Series(brands.columns)
brands.columns= brands.columns+s.groupby(s).cumcount().replace(0,'').astype(str) # idenitfied duplicated columns and numbered them
brands = brands.reindex(sorted(brands.columns), axis=1) 

In [56]:
cpgFlatId = pd.json_normalize(brands['cpg'], errors='ignore', record_prefix='cpg' , max_level= 1)\
    .add_prefix('cpgId') #unnesting by variable and adding prefix to column name for distinction

In [57]:
brandsClean = pd.merge(brands, cpgFlatId, left_index = True, right_index = True, how = 'outer')

In [58]:
brandsClean = brandsClean.drop(['cpg'], axis= 1)
brandsClean.rename(columns =  {'cpg1': 'cpgRef', '_id':'brandId'}, inplace= True)
brandsClean.rename(columns=lambda x: x.split('$')[0].replace(' ','') if '$' in x else x, inplace= True) #removing json keys in column name

In [59]:
brandsClean = brandsClean.apply(strip_obj, axis=0)
brandsClean = brandsClean.replace('', np.nan, regex=True) # converted empty spaces into nulls

In [60]:
brandsClean = brandsClean.dropna(axis = 1, how = 'all')
brandsClean = brandsClean.drop_duplicates()

## UNIX Timestamp to DateTime

Now that we have flatten out all the JSON objects in our datasets, we will now convert the unix/epoch time that are present in some of our variables into a regular time stamp. For this purpose, we are going to define a function that divides our unix epoch by 1000 and convert our seconds to a UTC timestamp. From there we will use pandas.to_datetime() to convert our timestamps to datetime types. This method will apply to both the user and receipts datasets. 


In [61]:
def dateConverter(x):
  try:
    return(datetime.utcfromtimestamp(x/1000).strftime('%Y-%m-%d %H:%M:%S'))
  except:
    return pd.NaT #coerce non integers to NaT

In [62]:
for col in receiptsClean.columns:
    if 'date' in col.lower():
        receiptsClean[col] = receiptsClean[col].apply(lambda date: dateConverter(date)).apply(pd.to_datetime) # applied to convert data to datetime

In [63]:
users = users.reindex(sorted(users.columns), axis=1) 

In [64]:
for col in users.columns:
    if any(x in col.lower() for x in ['date', 'login']):
        users[col] = users[col].apply(lambda date: dateConverter(date)).apply(pd.to_datetime)

In [65]:
usersClean = users.apply(strip_obj, axis=0)
usersClean.rename(columns = {'_id':'userId'}, inplace= True)

In [66]:
usersClean = usersClean.drop_duplicates()
usersClean = usersClean.dropna(axis = 1, how = 'all')

## Table Designing in SQL

In order to insert our data into a database that we can later leverage for analysis, we need to specify the datatypes that each of our columns will contain. Otherwise, pandas.to_sql() will just assume everything is nvarchar(max) which greatly affects performance and data integrity. To do this, we are going to define a function that utilizes the datatypes of our pandas.dataframes and updates them to sqlalchemy datatypes. From there, we will pass our columns to our dtype argument which will resolve our nvarchar(max) issue and make our tables usable for querying. 

In [67]:
def url(server_name, database): # Defined function to create url used for creating engines
    return(f'mssql+pyodbc://{server_name}/{database}?driver=ODBC Driver 17 for SQL Server')

In [68]:
fr = url('LAPTOP-9Q779EDT','fetchRewards')
engine = sal.create_engine(fr)
conn = engine.connect()

In [69]:
%%capture
usersClean.info()
receiptsClean.info()
brandsClean.info()

In [70]:
for col in receiptsClean.columns: # Converting strings to float 
    if any(x in col.lower() for x in ['price', 'total']):
        receiptsClean[col] = receiptsClean[col].astype(float)

In [71]:
receiptsFloat = ['pointsEarned','pointsEarnedReceipt' ]
receiptsClean[receiptsFloat] = receiptsClean[receiptsFloat].astype(float)

In [72]:
def sqlcol(df, dtypedict = None):    # To use dtype with to_sql(), we pass a dictionary keyed to each data frame column with corresponding sqlalchemy types.
    dtypedict = {}
    for i,j in zip(df.columns, df.dtypes):
        if "object" in str(j):
            dtypedict.update({i: sal.types.VARCHAR(length=255)})                       
        if "datetime" in str(j):
            dtypedict.update({i: sal.types.DateTime()})
        if "float" in str(j):
            dtypedict.update({i: sal.types.Float(precision=3, asdecimal=True)})
        if "int" in str(j):
            dtypedict.update({i: sal.types.INT()})
        if "bool" in str(j):
            dtypedict.update({i: sal.types.VARCHAR(length=255)})
    return dtypedict

In [73]:
usersType = sqlcol(usersClean)
usersClean.to_sql('users', conn, if_exists='replace', index=False, chunksize =5000, dtype = usersType); #Created a new table if it does not exist otherwise drop before inserting data frame values to this table. 
# Chunksize is used to specify the number of rows to be wrriten in each batch

In [74]:
receiptsType =sqlcol(receiptsClean)
receiptsClean.to_sql('receipts', conn, if_exists='replace', index=False, chunksize= 5000, dtype = receiptsType);

In [75]:
brandsType =sqlcol(brandsClean)
brandsClean.to_sql('brands', conn, if_exists='replace', index=False, chunksize= 5000, dtype = brandsType);

---
# Part Two: Write a query that directly answers a predetermined question from a business stakeholder

In this section, we will anwser several predetermined questions using our newly developed tables and ensure that additional considerations overlooked are taken into account. 

## Querying our Data

To find out which brands have the most spend and transactions among users that were created within the past 6 months, we first need to define a date that we can use as reference for limiting the scope of our users. From there, we join this table with our receipts table and then group our brands by the sum of our 'totalSpent' and 'purchasedItemCount' to get our desired fields. Finally, we will use this data to create visuals using ThinkCell in Microsoft Powerpoint to define the overall spending and transaction share of the top five brands compared to the rest of the brands.


In [76]:
brandsSpendTrans= pd.DataFrame(pd.read_sql_query("""\
DECLARE @createdDate DATETIME = (SELECT MAX(createdDate) FROM users);

with receipt_uniq as
	(SELECT DISTINCT receiptId,brandCode,totalSpent, purchasedItemCount, userId
	FROM receipts), 
top5 as 
	(SELECT DISTINCT brandCode
		,SUM(totalSpent) OVER(PARTITION BY brandCode) [spend]
		,SUM(purchasedItemCount) OVER(PARTITION BY brandCode)[transactions]
	FROM receipt_uniq

	INNER JOIN
	(SELECT userId, createdDate
	FROM users
	WHERE DATEDIFF(MONTH, createdDate, @createdDate) between 0 and 6 )
	users on users.userId = receipt_uniq.userId)

SELECT brandCode, 
ROUND([spend],1) [spend], 
ROUND([transactions],1) [transactions]
FROM top5
ORDER BY transactions desc
""", conn))

In [77]:
print("Top five brands by spend among users who were created within the past 6 months")
top5 = brandsSpendTrans.dropna() # drop null to accurately depict top brands in our tables below
top5.nlargest(5,'spend')[['brandCode', 'spend']]

Top five brands by spend among users who were created within the past 6 months


,brandCode,spend
1,BEN AND JERRYS,20040.77
2,PEPSI,19390.63
4,FOLGERS,18900.04
6,KELLOGG'S,18555.40
7,BIGELOW,18362.71


In [78]:
print("Top five brands by transactions among users who were created within the past 6 months")
top5.nlargest(5,'transactions')[['brandCode', 'transactions']]

Top five brands by transactions among users who were created within the past 6 months


,brandCode,transactions
1,BEN AND JERRYS,4111.0
2,PEPSI,3937.0
3,DOLE,3748.0
4,FOLGERS,3681.0
5,KNORR,3674.0


---
# Part Three: Evaluate Data Quality Issues in the Data Provided

In this section, we will look at one data quality issue that is of upmost importance to address in order to establish proper relational integrity.

## Examining the Relationship Between 'brandCodes' in the Brands and Receipts Table.

Part of the reason why developing a proper relational database with primary and foreign keys was not feasible was because there was a lot of missing data between the users, receipts, and brands datasets. In fact, without having proper knowledge of the API process and/or the schemas, it became quite difficult to remove fields without actually affecting the overall integrity of the data. Thus, I focused on having a minimal viable product that could allow me to extract meaningful insights while at the same time ensuring that the analysis would not end up skewed or inaccurate. 

### Brands

With this being said, one of the issues that needs to be tackled is the weak relationship that exists between the 'brandCode' field in the brands and receipts datasets. Starting with brands, I first looked at the overall counts of the brandCode field, noticed that there was more overall brandCodes than distinct ones (which shouldn't be the case given they should all be unique), and then followed up by dissecting 'nulls', 'test brands', and joinable values. In the end, when removing null and 'test brands', we are left with a little less than half of our values being joinable.


In [79]:
brandsResults= pd.DataFrame(pd.read_sql_query("""\
SELECT COUNT([brandCode]) [Total Brands], 
    [Total Distinct Brands], 
    [Total Nulls], 
    [Total Test Brands], 
    [Total Joinable Brands]
FROM brands

cross join 
(SELECT COUNT(DISTINCT [brandCode]) [Total Distinct Brands]
FROM brands) Dist

cross join
(SELECT SUM(CASE WHEN brandCode IS NULL THEN 1 ELSE 0 END) [Total Nulls]
FROM brands) nulls

cross join
(SELECT COUNT(DISTINCT brandCode) [Total Test Brands]
FROM brands
where brandCode like 'test%') test

cross join
(SELECT COUNT(DISTINCT brandCode) [Total Joinable Brands]
FROM brands
where brandCode not like 'test%' and brandCode is not null ) actual

GROUP BY [Total Distinct Brands], [Total Nulls], [Total Test Brands],[Total Joinable Brands];
""", conn))
print("Count of values in 'brandCode' field in brands")
brandsResults

Count of values in 'brandCode' field in brands


,Total Brands,Total Distinct Brands,Total Nulls,Total Test Brands,Total Joinable Brands
0,898,895,269,359,536


### Receipts

Since out receipts table is formatted by individual items, the number of brands listed is quite larger, however, when looking at distinct brands the number dramatically decreases because a majority of them are either duplicates or nulls. This poses a greater challenege because this limits our total joinable values to only 227. 

In [80]:
receiptsResults= pd.DataFrame(pd.read_sql_query("""\
SELECT COUNT([brandCode]) [Total Receipt Brands], 
    [Total Distinct Receipt Brands], 
    [Total Receipt Nulls], 
    [Total Joinable Receipt Brands]
FROM receipts

cross join 
(SELECT COUNT(DISTINCT [brandCode]) [Total Distinct Receipt Brands]
FROM receipts) Dist

cross join
(SELECT SUM(CASE WHEN brandCode IS NULL THEN 1 ELSE 0 END) [Total Receipt Nulls]
FROM receipts) nulls

cross join
(SELECT COUNT(DISTINCT brandCode) [Total Joinable Receipt Brands]
FROM receipts
where brandCode is not null) actual

GROUP BY [Total Distinct Receipt Brands], [Total Receipt Nulls], [Total Joinable Receipt Brands];
""", conn))
print("Count of values in 'brandCode' field in receipts")
receiptsResults

Count of values in 'brandCode' field in receipts


,Total Receipt Brands,Total Distinct Receipt Brands,Total Receipt Nulls,Total Joinable Receipt Brands
0,2600,227,4781,227


### Matching brandCode 

Given the number of nulls and 'test brands' found in both our datasets, the total number of matching brands only comes down to a staggering 41 brands. This, of course, greatly limits our ability to develop a proper relationship database and by extension extract meaning insights. 

In [81]:
matchResults= pd.DataFrame(pd.read_sql_query("""\
SELECT COUNT(DISTINCT brands.brandCode) as [Matching Brands]
FROM brands

INNER JOIN
(SELECT DISTINCT brandCode
FROM receipts) 
receipts ON receipts.brandCode = brands.brandCode;
""", conn))
print("Count of matching brands between brands and receipts table")
matchResults

Count of matching brands between brands and receipts table


,Matching Brands
0,41
